In [1]:
import os
import warnings
import importlib
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc, confusion_matrix, f1_score, accuracy_score, \
    precision_score, recall_score
import xgboost as xgb
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc, confusion_matrix, f1_score
from sklearn.model_selection import KFold, LeavePGroupsOut
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from matplotlib import pyplot as plt
import matplotlib
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from tabpfn import TabPFNClassifier
from sklearn.preprocessing import label_binarize
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.utils.class_weight import compute_sample_weight

%matplotlib inline
# matplotlib.use('TkAgg')
from xgboost import XGBClassifier
from imblearn.over_sampling import RandomOverSampler  # install via: pip install imbalanced-learn

warnings.filterwarnings('ignore')
import torch
from scipy.stats import skew
import lightgbm as lgb
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

seed = 69
torch.manual_seed(seed)
np.random.seed(seed)

import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


True
NVIDIA GeForce RTX 3080 Ti Laptop GPU


In [2]:
root = '/home/ali/PycharmProjects/tihm/dataset'

dataset_06h = pd.read_csv(os.path.join(root, 'data-06h.csv'))
dataset_12h = pd.read_csv(os.path.join(root, 'data-12h.csv'))
dataset_24h = pd.read_csv(os.path.join(root, 'data-24h.csv'))

In [3]:
# display(dataset_06h)
# display(dataset_12h)
# display(dataset_24h)

In [4]:
# print(dataset_06h['agitation'].equals(dataset_06h['agitation-next']))
# print(np.sum(dataset_06h['agitation']))
# print(np.sum(dataset_06h['agitation-next']))

# print(np.where(dataset_06h['agitation'] == 1))
# print(np.where(dataset_06h['agitation-next'] == 1))

print(np.sum(dataset_06h['agitation-four'] == 0),
      np.sum(dataset_06h['agitation-four'] == 1),
      np.sum(dataset_06h['agitation-four'] == 2),
      np.sum(dataset_06h['agitation-four'] == 3))

print(np.sum(dataset_06h['agitation-next'] == 0),
      np.sum(dataset_06h['agitation-next'] == 1))


1400 117 89 18
1752 135


In [10]:
# dataset = dataset_06h
dataset = dataset_12h
# label = 'agitation'
label = 'agitation-next'
# label = 'agitation-four'

y = np.array(dataset[[label]]).squeeze()

# Binary
y[y == -1] = 0
y[y >= 1] = 1

# QUATERNARY
# y[y == -1] = 0
# y[y == -10] = 0

dataset.drop(['agitation',
              'agitation-next',
              'agitation-four'],
              axis=1, inplace=True)

ids = np.array(dataset['id']).squeeze()
p = np.unique(ids, return_inverse=True)[1]

print(np.isnan(y).sum())
y = np.nan_to_num(y, nan=0)
print(np.isnan(y).sum())
print(np.isnan(p).sum())

print(np.unique(y))
print(y.sum())
display(y)
display(p)
# display(dataset.columns)

KeyError: "None of [Index(['agitation-next'], dtype='object')] are in the [columns]"

In [11]:
columns = dataset.columns
dataset = dataset[columns]
display(dataset)
print(dataset.columns.values)

,id,date,12h,back-door,bathroom,bedroom,fridge-door,front-door,hallway,kitchen,...,heart-rate,muscle-mass,systolic-blood-pressure,total-body-water,skin-temperature,blood-pressure,body-water,pulse,weight,body-temperature-label
0,0697d,2019-06-28,00-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
1,0697d,2019-06-28,12-24,14.0,7.0,24.0,23.0,28.0,40.0,106.0,...,42.0,64.5,165.0,50.6,NaN,1.0,0.0,1.0,0.0,0.0
2,0697d,2019-06-29,00-12,0.0,7.0,17.0,0.0,8.0,22.0,36.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
3,0697d,2019-06-29,12-24,2.0,4.0,9.0,8.0,15.0,35.0,84.0,...,60.0,64.6,157.0,51.2,NaN,1.0,0.0,0.0,0.0,0.0
4,0697d,2019-06-30,00-12,2.0,14.0,43.0,0.0,2.0,22.0,38.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5376,fd100,2019-06-28,00-12,0.0,22.0,41.0,0.0,16.0,31.0,52.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5377,fd100,2019-06-28,12-24,0.0,10.0,50.0,7.0,5.0,27.0,93.0,...,61.0,NaN,138.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5378,fd100,2019-06-29,00-12,0.0,10.0,21.0,7.0,3.0,13.0,32.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5379,fd100,2019-06-29,12-24,0.0,23.0,35.0,20.0,12.0,48.0,78.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


['id' 'date' '12h' 'back-door' 'bathroom' 'bedroom' 'fridge-door'
 'front-door' 'hallway' 'kitchen' 'lounge' 'total-events'
 'unique-locations' 'active-location-ratio' 'private-to-public-ratio'
 'location-entropy' 'location-dominance-ratio' 'back-and-forth-count'
 'num-transitions' 'back-door-count-max' 'back-door-count-mean'
 'back-door-count-std' 'back-door-count-sum' 'bathroom-count-max'
 'bathroom-count-mean' 'bathroom-count-std' 'bathroom-count-sum'
 'bedroom-count-max' 'bedroom-count-mean' 'bedroom-count-std'
 'bedroom-count-sum' 'fridge-door-count-max' 'fridge-door-count-mean'
 'fridge-door-count-std' 'fridge-door-count-sum' 'front-door-count-max'
 'front-door-count-mean' 'front-door-count-std' 'front-door-count-sum'
 'hallway-count-max' 'hallway-count-mean' 'hallway-count-std'
 'hallway-count-sum' 'kitchen-count-max' 'kitchen-count-mean'
 'kitchen-count-std' 'kitchen-count-sum' 'lounge-count-max'
 'lounge-count-mean' 'lounge-count-std' 'lounge-count-sum'
 'body-temperature' 'bo

In [12]:
columns_count = ['back-door', 'bathroom', 'bedroom', 'fridge-door',
                 'front-door', 'hallway', 'kitchen', 'lounge']

columns_contextual = ['total-events', 'unique-locations', 'active-location-ratio', 'private-to-public-ratio',
                      'location-entropy', 'location-dominance-ratio', 'back-and-forth-count', 'num-transitions']

columns_statistical = ['back-door-count-max', 'back-door-count-mean', 'back-door-count-std', 'back-door-count-sum',
                       'bathroom-count-max', 'bathroom-count-mean', 'bathroom-count-std', 'bathroom-count-sum',
                       'bedroom-count-max', 'bedroom-count-mean', 'bedroom-count-std', 'bedroom-count-sum',
                       'fridge-door-count-max', 'fridge-door-count-mean', 'fridge-door-count-std', 'fridge-door-count-sum',
                       'front-door-count-max', 'front-door-count-mean', 'front-door-count-std', 'front-door-count-sum',
                       'hallway-count-max', 'hallway-count-mean', 'hallway-count-std', 'hallway-count-sum',
                       'kitchen-count-max', 'kitchen-count-mean', 'kitchen-count-std','kitchen-count-sum',
                       'lounge-count-max', 'lounge-count-mean', 'lounge-count-std','lounge-count-sum']

columns_physiology = ['body-temperature', 'body-weight', 'diastolic-blood-pressure', 'heart-rate',
                      'muscle-mass', 'systolic-blood-pressure', 'total-body-water', 'skin-temperature',
                      'blood-pressure', 'body-water', 'pulse', 'weight',
                      'body-temperature-label']

print(len(columns_count),
      len(columns_contextual),
      len(columns_statistical),
      len(columns_physiology))

8 8 32 13


In [13]:
# Imputation
import utils_data
importlib.reload(utils_data)
from utils_data import hierarchical_imputation

print(dataset.isna().sum().sum())

dataset = hierarchical_imputation(dataset)

print(dataset.isna().sum().sum())

dataset.drop([
    'id',
    'date',
    '12h',
    ], axis=1, inplace=True)

62400
0


In [14]:
# Ablation
dataset_CNT_STA_CTX_PHY = dataset
dataset_CNT_STA_CTX_PHY.to_csv(os.path.join(root,   'dataset_12h_imputed_CNT_STA_CTX_PHY.csv'), index=False)
print(dataset_CNT_STA_CTX_PHY.shape)

dataset_CNT_STA_CTX = dataset.drop(columns_physiology, axis=1, inplace=False)
dataset_CNT_STA_CTX.to_csv(os.path.join(root,       'dataset_12h_imputed_CNT_STA_CTX.csv'), index=False)
print(dataset_CNT_STA_CTX.shape)


dataset_CNT_STA = dataset.drop(columns_contextual + columns_physiology, axis=1, inplace=False)
dataset_CNT_STA.to_csv(os.path.join(root,           'dataset_12h_imputed_CNT_STA.csv'), index=False)
print(dataset_CNT_STA.shape)

dataset_CNT = dataset.drop(columns_statistical + columns_contextual + columns_physiology, axis=1, inplace=False)
dataset_CNT.to_csv(os.path.join(root,           'dataset_12h_imputed_CNT.csv'), index=False)
print(dataset_CNT.shape)

print(dataset.columns.values)

(5381, 61)
(5381, 48)
(5381, 40)
(5381, 8)
['back-door' 'bathroom' 'bedroom' 'fridge-door' 'front-door' 'hallway'
 'kitchen' 'lounge' 'total-events' 'unique-locations'
 'active-location-ratio' 'private-to-public-ratio' 'location-entropy'
 'location-dominance-ratio' 'back-and-forth-count' 'num-transitions'
 'back-door-count-max' 'back-door-count-mean' 'back-door-count-std'
 'back-door-count-sum' 'bathroom-count-max' 'bathroom-count-mean'
 'bathroom-count-std' 'bathroom-count-sum' 'bedroom-count-max'
 'bedroom-count-mean' 'bedroom-count-std' 'bedroom-count-sum'
 'fridge-door-count-max' 'fridge-door-count-mean' 'fridge-door-count-std'
 'fridge-door-count-sum' 'front-door-count-max' 'front-door-count-mean'
 'front-door-count-std' 'front-door-count-sum' 'hallway-count-max'
 'hallway-count-mean' 'hallway-count-std' 'hallway-count-sum'
 'kitchen-count-max' 'kitchen-count-mean' 'kitchen-count-std'
 'kitchen-count-sum' 'lounge-count-max' 'lounge-count-mean'
 'lounge-count-std' 'lounge-count-sum

In [ ]:
# print(y.shape)
# display(dataset)

# dataset.to_csv(os.path.join(root, 'dataset_06h_imputed.csv'))
# np.save('dataset_06h_y.npy', y)

In [18]:
# Dataset

dataset = pd.read_csv(os.path.join(root, 'dataset_12h_imputed_CNT_STA_CTX_PHY.csv'))
# dataset = pd.read_csv(os.path.join(root, 'dataset_12h_imputed_CNT.csv'))
# dataset.drop(['Unnamed: 0'], axis=1, inplace=True)
# y = np.load('dataset_06h_y.npy')

dataset = dataset[dataset.columns]
x = np.array(dataset)


print(dataset.columns.values)
print(y.shape)
print(x.shape)
print(np.unique(y))
display(dataset)

['back-door' 'bathroom' 'bedroom' 'fridge-door' 'front-door' 'hallway'
 'kitchen' 'lounge' 'total-events' 'unique-locations'
 'active-location-ratio' 'private-to-public-ratio' 'location-entropy'
 'location-dominance-ratio' 'back-and-forth-count' 'num-transitions'
 'back-door-count-max' 'back-door-count-mean' 'back-door-count-std'
 'back-door-count-sum' 'bathroom-count-max' 'bathroom-count-mean'
 'bathroom-count-std' 'bathroom-count-sum' 'bedroom-count-max'
 'bedroom-count-mean' 'bedroom-count-std' 'bedroom-count-sum'
 'fridge-door-count-max' 'fridge-door-count-mean' 'fridge-door-count-std'
 'fridge-door-count-sum' 'front-door-count-max' 'front-door-count-mean'
 'front-door-count-std' 'front-door-count-sum' 'hallway-count-max'
 'hallway-count-mean' 'hallway-count-std' 'hallway-count-sum'
 'kitchen-count-max' 'kitchen-count-mean' 'kitchen-count-std'
 'kitchen-count-sum' 'lounge-count-max' 'lounge-count-mean'
 'lounge-count-std' 'lounge-count-sum' 'body-temperature' 'body-weight'
 'diasto

,back-door,bathroom,bedroom,fridge-door,front-door,hallway,kitchen,lounge,total-events,unique-locations,...,heart-rate,muscle-mass,systolic-blood-pressure,total-body-water,skin-temperature,blood-pressure,body-water,pulse,weight,body-temperature-label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,72.1250,47.5853,136.4271,49.8206,34.2843,0.0000,0.0000,0.0000,0.0000,0.0
1,14.0,7.0,24.0,23.0,28.0,40.0,106.0,80.0,322.0,8.0,...,42.0000,64.5000,165.0000,50.6000,34.2843,1.0000,0.0000,1.0000,0.0000,0.0
2,0.0,7.0,17.0,0.0,8.0,22.0,36.0,18.0,108.0,6.0,...,75.7273,48.6600,130.7424,51.7600,34.5600,0.0000,0.0000,0.0000,0.0000,0.0
3,2.0,4.0,9.0,8.0,15.0,35.0,84.0,99.0,256.0,8.0,...,60.0000,64.6000,157.0000,51.2000,34.5600,1.0000,0.0000,0.0000,0.0000,0.0
4,2.0,14.0,43.0,0.0,2.0,22.0,38.0,25.0,146.0,7.0,...,67.4375,45.3364,136.6875,50.1000,34.3144,0.0000,0.0000,0.0000,0.0000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5376,0.0,22.0,41.0,0.0,16.0,31.0,52.0,45.0,207.0,6.0,...,72.1250,47.5853,136.4271,49.8206,34.2843,0.2222,0.0000,0.1667,0.0000,0.0
5377,0.0,10.0,50.0,7.0,5.0,27.0,93.0,75.0,267.0,7.0,...,61.0000,47.5853,138.0000,49.8206,34.2843,0.2222,0.0000,0.1667,0.0000,0.0
5378,0.0,10.0,21.0,7.0,3.0,13.0,32.0,16.0,102.0,7.0,...,75.7273,48.6600,130.7424,51.7600,34.5600,0.3125,0.0625,0.3750,0.0625,0.0
5379,0.0,23.0,35.0,20.0,12.0,48.0,78.0,80.0,296.0,7.0,...,75.7273,48.6600,130.7424,51.7600,34.5600,0.3125,0.0625,0.3750,0.0625,0.0


In [25]:
# LightGBM

Y_TRUES = np.empty([0])
Y_PROBS = np.empty([0])
Y_PREDS = np.empty([0])


# cv = LeavePGroupsOut(n_groups=1)
# for train_idx, test_idx in cv.split(x, y, groups=p):
#     participant = np.unique(p[test_idx])[0]


# cv = KFold(n_splits=x.shape[0], shuffle=True, random_state=seed)
# for fold, (train_idx, test_idx) in enumerate(cv.split(x), start=1):
#     participant = np.unique(p[test_idx])[0]

cv = KFold(n_splits=5, shuffle=True, random_state=seed)
for fold, (train_idx, test_idx) in enumerate(cv.split(x), start=1):
    participant = fold

    x_train, x_test = x[train_idx], x[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    print(participant, x_train.shape[0], x_test.shape[0])

    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)

    normalizer = MinMaxScaler()
    normalizer.fit(x_train)
    x_train = normalizer.transform(x_train)
    x_test = normalizer.transform(x_test)

    # Oversample class 1

    # ros = RandomOverSampler(sampling_strategy=.1, random_state=seed)
    # x_train, y_train = ros.fit_resample(x_train, y_train)

    # 1. Check original class distribution
    # print("Original class distribution:", Counter(y_train))
    # # 2. Define the minority class (adjust if needed)
    # minority_class = 1  # change this if your minority class label is different
    # current_minority_count = sum(y_train == minority_class)
    # # 3. Define desired new total count for the minority class (10x)
    # target_minority_count = current_minority_count * 10
    # # 4. Setup SMOTE with custom sampling strategy
    # smote = SMOTE(sampling_strategy={minority_class: target_minority_count}, random_state=seed)
    # # 5. Fit and resample
    # x_train, y_train = smote.fit_resample(x_train, y_train)
    # # 6. Confirm new distribution
    # print("Resampled class distribution:", Counter(y_train))



    # ----- LightGBM
    train_data = lgb.Dataset(x_train, label=y_train)
    test_data = lgb.Dataset(x_test, label=y_test, reference=train_data)
    params = {
        'verbose': -1,  # 👈 turn off training output
        'objective': 'binary',
        'metric': 'binary_logloss',  # or 'auc' if you prefer
        'num_leaves': 64,
        'learning_rate': 0.01,
        'n_estimators': 100,
        # 'is_unbalance': True  # Automatically balances positive and negative classes
        # 'scale_pos_weight': (y_train == 0).sum() / (y_train == 1).sum()
    }
    bst = lgb.train(params, train_data, valid_sets=[train_data, test_data])
    y_probs = bst.predict(x_test, num_iteration=bst.best_iteration)


    # ----- Transformer
    # model = TabPFNClassifier(
    #     device='cuda',
    #     random_state=seed,
    #     n_estimators = 4,
    #     ignore_pretraining_limits=True)
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]


    Y_TRUES = np.append(Y_TRUES, y_test)
    Y_PROBS = np.append(Y_PROBS, y_probs)
    Y_PREDS = np.append(Y_PREDS, (y_probs >= .5).astype(int))


indx = Y_TRUES.argsort()
Y_TRUES = Y_TRUES[indx]
Y_PROBS = Y_PROBS[indx]
Y_PREDS = Y_PREDS[indx]


# Evaluation metrics
auc_roc = roc_auc_score(Y_TRUES, Y_PROBS)
precision, recall, _ = precision_recall_curve(Y_TRUES, Y_PROBS)
auc_pr = auc(recall, precision)
acc = accuracy_score(Y_TRUES, Y_PREDS)
pre = precision_score(Y_TRUES, Y_PREDS)
rec = recall_score(Y_TRUES, Y_PREDS)
f1 = f1_score(Y_TRUES, Y_PREDS)

# Confusion matrix: [ [TN, FP], [FN, TP] ]
tn, fp, fn, tp = confusion_matrix(Y_TRUES, Y_PREDS).ravel()

# Sensitivity = Recall = TP / (TP + FN)
sensitivity = tp / (tp + fn)

# Specificity = TN / (TN + FP)
specificity = tn / (tn + fp)

# Print results
print(f"AUC-ROC:    {auc_roc:.4f}")
print(f"AUC-PR:     {auc_pr:.4f}")
print(f"Accuracy:   {acc:.4f}")
print(f"Precision:  {pre:.4f}")
print(f"Recall:     {rec:.4f}")
print(f"F1 Score:   {f1:.4f}")
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity:          {specificity:.4f}")
print(confusion_matrix(Y_TRUES, Y_PREDS))


# plt.figure()
# plt.plot(Y_TRUES, 'o', color='blue', alpha=.25, markersize=8, label='Ground-truth')
# plt.plot(Y_PREDS, 'o', color='red', alpha=.25, markersize=8, label='Prediction')
# plt.show()

1 8632 2158
2 8632 2158
3 8632 2158
4 8632 2158
5 8632 2158
AUC-ROC:    0.9544
AUC-PR:     0.4889
Accuracy:   0.9886
Precision:  0.8750
Recall:     0.1037
F1 Score:   0.1854
Sensitivity (Recall): 0.1037
Specificity:          0.9998
[[10653     2]
 [  121    14]]


In [221]:
# Transformer

Y_TRUES = np.empty([0])
Y_PROBS = np.empty([0])
Y_PREDS = np.empty([0])


cv = LeavePGroupsOut(n_groups=1)
for train_idx, test_idx in cv.split(x, y, groups=p):
    participant = np.unique(p[test_idx])[0]


# cv = KFold(n_splits=x.shape[0], shuffle=True, random_state=seed)
# for fold, (train_idx, test_idx) in enumerate(cv.split(x), start=1):
#     participant = np.unique(p[test_idx])[0]

# cv = KFold(n_splits=5, shuffle=True, random_state=seed)
# for fold, (train_idx, test_idx) in enumerate(cv.split(x), start=1):
#     participant = fold

    x_train, x_test = x[train_idx], x[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    print(participant, x_train.shape[0], x_test.shape[0])

    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)

    normalizer = MinMaxScaler()
    normalizer.fit(x_train)
    x_train = normalizer.transform(x_train)
    x_test = normalizer.transform(x_test)

    # Oversample class 1

    # ros = RandomOverSampler(sampling_strategy=.1, random_state=seed)
    # x_train, y_train = ros.fit_resample(x_train, y_train)

    # # 1. Check original class distribution
    # print("Original class distribution:", Counter(y_train))
    # # 2. Define the minority class (adjust if needed)
    # minority_class = 1  # change this if your minority class label is different
    # current_minority_count = sum(y_train == minority_class)
    # # 3. Define desired new total count for the minority class (10x)
    # target_minority_count = current_minority_count * 10
    # # 4. Setup SMOTE with custom sampling strategy
    # smote = SMOTE(sampling_strategy={minority_class: target_minority_count}, random_state=seed)
    # # 5. Fit and resample
    # x_train, y_train = smote.fit_resample(x_train, y_train)
    # # 6. Confirm new distribution
    # print("Resampled class distribution:", Counter(y_train))



    # ----- LightGBM
    # train_data = lgb.Dataset(x_train, label=y_train)
    # test_data = lgb.Dataset(x_test, label=y_test, reference=train_data)
    # params = {
    #     'verbose': -1,  # 👈 turn off training output
    #     'objective': 'binary',
    #     'metric': 'binary_logloss',  # or 'auc' if you prefer
    #     'num_leaves': 64,
    #     'learning_rate': 0.01,
    #     'n_estimators': 100,
    #     # 'is_unbalance': True  # Automatically balances positive and negative classes
    #     # 'scale_pos_weight': (y_train == 0).sum() / (y_train == 1).sum()
    # }
    # bst = lgb.train(params, train_data, valid_sets=[train_data, test_data])
    # y_probs = bst.predict(x_test, num_iteration=bst.best_iteration)


    # ----- Transformer
    model = TabPFNClassifier(
        device='cuda',
        random_state=seed,
        n_estimators = 4,
        ignore_pretraining_limits=True)
    model.fit(x_train, y_train)
    y_probs = model.predict_proba(x_test)[:, 1]


    Y_TRUES = np.append(Y_TRUES, y_test)
    Y_PROBS = np.append(Y_PROBS, y_probs)
    Y_PREDS = np.append(Y_PREDS, (y_probs >= .5).astype(int))


indx = Y_TRUES.argsort()
Y_TRUES = Y_TRUES[indx]
Y_PROBS = Y_PROBS[indx]
Y_PREDS = Y_PREDS[indx]


# Evaluation metrics
auc_roc = roc_auc_score(Y_TRUES, Y_PROBS)
precision, recall, _ = precision_recall_curve(Y_TRUES, Y_PROBS)
auc_pr = auc(recall, precision)
acc = accuracy_score(Y_TRUES, Y_PREDS)
pre = precision_score(Y_TRUES, Y_PREDS)
rec = recall_score(Y_TRUES, Y_PREDS)
f1 = f1_score(Y_TRUES, Y_PREDS)

# Confusion matrix: [ [TN, FP], [FN, TP] ]
tn, fp, fn, tp = confusion_matrix(Y_TRUES, Y_PREDS).ravel()

# Sensitivity = Recall = TP / (TP + FN)
sensitivity = tp / (tp + fn)

# Specificity = TN / (TN + FP)
specificity = tn / (tn + fp)

# Print results
print(f"AUC-ROC:    {auc_roc:.4f}")
print(f"AUC-PR:     {auc_pr:.4f}")
print(f"Accuracy:   {acc:.4f}")
print(f"Precision:  {pre:.4f}")
print(f"Recall:     {rec:.4f}")
print(f"F1 Score:   {f1:.4f}")
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity:          {specificity:.4f}")
print(confusion_matrix(Y_TRUES, Y_PREDS))


# plt.figure()
# plt.plot(Y_TRUES, 'o', color='blue', alpha=.25, markersize=8, label='Ground-truth')
# plt.plot(Y_PREDS, 'o', color='red', alpha=.25, markersize=8, label='Prediction')
# plt.show()

0 10779 11
1 10617 173
2 10488 302
3 10599 191
4 10604 186
5 10767 23
6 10437 353
7 10541 249
8 10500 290
9 10611 179
10 10771 19
11 10560 230
12 10532 258
13 10716 74
14 10524 266
15 10548 242
16 10665 125
17 10508 282
18 10668 122
19 10570 220
20 10588 202
21 10521 269
22 10627 163
23 10600 190
24 10771 19
25 10520 270
26 10495 295
27 10520 270
28 10605 185
29 10616 174
30 10576 214
31 10556 234
32 10687 103
33 10560 230
34 10625 165
35 10751 39
36 10525 265
37 10436 354
38 10621 169
39 10463 327
40 10524 266
41 10584 206
42 10632 158
43 10520 270
44 10728 62
45 10699 91
46 10763 27
47 10580 210
48 10672 118
49 10528 262
50 10627 163
51 10541 249
52 10502 288
53 10575 215
54 10532 258
55 10775 15
AUC-ROC:    0.9730
AUC-PR:     0.3946
Accuracy:   0.9873
Precision:  0.4750
Recall:     0.1407
F1 Score:   0.2171
Sensitivity (Recall): 0.1407
Specificity:          0.9980
[[10634    21]
 [  116    19]]


In [19]:
# Gradient Boosting Classifier

Y_TRUES = np.empty([0])
Y_PROBS = np.empty([0])
Y_PREDS = np.empty([0])


# cv = LeavePGroupsOut(n_groups=1)
# for train_idx, test_idx in cv.split(x, y, groups=p):
#     participant = np.unique(p[test_idx])[0]


# cv = KFold(n_splits=x.shape[0], shuffle=True, random_state=seed)
# for fold, (train_idx, test_idx) in enumerate(cv.split(x), start=1):
#     participant = np.unique(p[test_idx])[0]

cv = KFold(n_splits=5, shuffle=True, random_state=seed)
for fold, (train_idx, test_idx) in enumerate(cv.split(x), start=1):
    participant = fold

    x_train, x_test = x[train_idx], x[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    print(participant, x_train.shape[0], x_test.shape[0])

    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)

    normalizer = MinMaxScaler()
    normalizer.fit(x_train)
    x_train = normalizer.transform(x_train)
    x_test = normalizer.transform(x_test)

    # Oversample class 1

    # ros = RandomOverSampler(sampling_strategy=.1, random_state=seed)
    # x_train, y_train = ros.fit_resample(x_train, y_train)

    # # 1. Check original class distribution
    # print("Original class distribution:", Counter(y_train))
    # # 2. Define the minority class (adjust if needed)
    # minority_class = 1  # change this if your minority class label is different
    # current_minority_count = sum(y_train == minority_class)
    # # 3. Define desired new total count for the minority class (10x)
    # target_minority_count = current_minority_count * 10
    # # 4. Setup SMOTE with custom sampling strategy
    # smote = SMOTE(sampling_strategy={minority_class: target_minority_count}, random_state=seed)
    # # 5. Fit and resample
    # x_train, y_train = smote.fit_resample(x_train, y_train)
    # # 6. Confirm new distribution
    # print("Resampled class distribution:", Counter(y_train))


    # ----- LightGBM
    # train_data = lgb.Dataset(x_train, label=y_train)
    # test_data = lgb.Dataset(x_test, label=y_test, reference=train_data)
    # params = {
    #     'verbose': -1,  # 👈 turn off training output
    #     'objective': 'binary',
    #     'metric': 'binary_logloss',  # or 'auc' if you prefer
    #     'num_leaves': 64,
    #     'learning_rate': 0.01,
    #     'n_estimators': 100,
    #     # 'is_unbalance': True  # Automatically balances positive and negative classes
    #     # 'scale_pos_weight': (y_train == 0).sum() / (y_train == 1).sum()
    # }
    # bst = lgb.train(params, train_data, valid_sets=[train_data, test_data])
    # y_probs = bst.predict(x_test, num_iteration=bst.best_iteration)


    # ----- Transformer
    # model = TabPFNClassifier(
    #     device='cuda',
    #     random_state=seed,
    #     n_estimators = 4,
    #     ignore_pretraining_limits=True)
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]


    # ----- Gradient Boosting Classifier
    model = GradientBoostingClassifier(
        n_estimators=100,  # match LightGBM
        learning_rate=0.001,  # match LightGBM
        max_depth=4,  # similar to LightGBM default tree depth
        subsample=1.0,  # default
        random_state=seed
    )
    # model.fit(x_train, y_train)
    model.fit(x_train, y_train, sample_weight=compute_sample_weight(class_weight='balanced', y=y_train))
    y_probs = model.predict_proba(x_test)[:, 1]
    

    Y_TRUES = np.append(Y_TRUES, y_test)
    Y_PROBS = np.append(Y_PROBS, y_probs)
    Y_PREDS = np.append(Y_PREDS, (y_probs >= .5).astype(int))


indx = Y_TRUES.argsort()
Y_TRUES = Y_TRUES[indx]
Y_PROBS = Y_PROBS[indx]
Y_PREDS = Y_PREDS[indx]


# Evaluation metrics
auc_roc = roc_auc_score(Y_TRUES, Y_PROBS)
precision, recall, _ = precision_recall_curve(Y_TRUES, Y_PROBS)
auc_pr = auc(recall, precision)
acc = accuracy_score(Y_TRUES, Y_PREDS)
pre = precision_score(Y_TRUES, Y_PREDS)
rec = recall_score(Y_TRUES, Y_PREDS)
f1 = f1_score(Y_TRUES, Y_PREDS)

# Confusion matrix: [ [TN, FP], [FN, TP] ]
tn, fp, fn, tp = confusion_matrix(Y_TRUES, Y_PREDS).ravel()

# Sensitivity = Recall = TP / (TP + FN)
sensitivity = tp / (tp + fn)

# Specificity = TN / (TN + FP)
specificity = tn / (tn + fp)

# Print results
print(f"AUC-ROC:    {auc_roc:.4f}")
print(f"AUC-PR:     {auc_pr:.4f}")
print(f"Accuracy:   {acc:.4f}")
print(f"Precision:  {pre:.4f}")
print(f"Recall:     {rec:.4f}")
print(f"F1 Score:   {f1:.4f}")
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity:          {specificity:.4f}")
print(confusion_matrix(Y_TRUES, Y_PREDS))


# plt.figure()
# plt.plot(Y_TRUES, 'o', color='blue', alpha=.25, markersize=8, label='Ground-truth')
# plt.plot(Y_PREDS, 'o', color='red', alpha=.25, markersize=8, label='Prediction')
# plt.show()

1 4304 1077
2 4305 1076
3 4305 1076
4 4305 1076
5 4305 1076
AUC-ROC:    0.8900
AUC-PR:     0.2280
Accuracy:   0.9205
Precision:  0.1953
Recall:     0.8319
F1 Score:   0.3163
Sensitivity (Recall): 0.8319
Specificity:          0.9225
[[4854  408]
 [  20   99]]


In [43]:
# XGBoost

Y_TRUES = np.empty([0])
Y_PROBS = np.empty([0])
Y_PREDS = np.empty([0])


cv = LeavePGroupsOut(n_groups=1)
for train_idx, test_idx in cv.split(x, y, groups=p):
    participant = np.unique(p[test_idx])[0]


# cv = KFold(n_splits=x.shape[0], shuffle=True, random_state=seed)
# for fold, (train_idx, test_idx) in enumerate(cv.split(x), start=1):
#     participant = np.unique(p[test_idx])[0]

# cv = KFold(n_splits=5, shuffle=True, random_state=seed)
# for fold, (train_idx, test_idx) in enumerate(cv.split(x), start=1):
#     participant = fold

    x_train, x_test = x[train_idx], x[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    print(participant, x_train.shape[0], x_test.shape[0])

    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)

    normalizer = MinMaxScaler()
    normalizer.fit(x_train)
    x_train = normalizer.transform(x_train)
    x_test = normalizer.transform(x_test)

    # Oversample class 1

    # ros = RandomOverSampler(sampling_strategy=.1, random_state=seed)
    # x_train, y_train = ros.fit_resample(x_train, y_train)

    # # 1. Check original class distribution
    # print("Original class distribution:", Counter(y_train))
    # # 2. Define the minority class (adjust if needed)
    # minority_class = 1  # change this if your minority class label is different
    # current_minority_count = sum(y_train == minority_class)
    # # 3. Define desired new total count for the minority class (10x)
    # target_minority_count = current_minority_count * 10
    # # 4. Setup SMOTE with custom sampling strategy
    # smote = SMOTE(sampling_strategy={minority_class: target_minority_count}, random_state=seed)
    # # 5. Fit and resample
    # x_train, y_train = smote.fit_resample(x_train, y_train)
    # # 6. Confirm new distribution
    # print("Resampled class distribution:", Counter(y_train))


    # ----- LightGBM
    # train_data = lgb.Dataset(x_train, label=y_train)
    # test_data = lgb.Dataset(x_test, label=y_test, reference=train_data)
    # params = {
    #     'verbose': -1,  # 👈 turn off training output
    #     'objective': 'binary',
    #     'metric': 'binary_logloss',  # or 'auc' if you prefer
    #     'num_leaves': 64,
    #     'learning_rate': 0.01,
    #     'n_estimators': 100,
    #     # 'is_unbalance': True  # Automatically balances positive and negative classes
    #     # 'scale_pos_weight': (y_train == 0).sum() / (y_train == 1).sum()
    # }
    # bst = lgb.train(params, train_data, valid_sets=[train_data, test_data])
    # y_probs = bst.predict(x_test, num_iteration=bst.best_iteration)


    # ----- Transformer
    # model = TabPFNClassifier(
    #     device='cuda',
    #     random_state=seed,
    #     n_estimators = 4,
    #     ignore_pretraining_limits=True)
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]


    # ----- Gradient Boosting Classifier
    # model = GradientBoostingClassifier(
    #     n_estimators=100,  # match LightGBM
    #     learning_rate=0.001,  # match LightGBM
    #     max_depth=4,  # similar to LightGBM default tree depth
    #     subsample=1.0,  # default
    #     random_state=seed
    # )
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]
    

    # ----- XGBoost
    model = XGBClassifier()
    # model.fit(x_train, y_train)
    model.fit(x_train, y_train, sample_weight=compute_sample_weight(class_weight='balanced', y=y_train))
    y_probs = model.predict_proba(x_test)[:, 1]


    Y_TRUES = np.append(Y_TRUES, y_test)
    Y_PROBS = np.append(Y_PROBS, y_probs)
    Y_PREDS = np.append(Y_PREDS, (y_probs >= .5).astype(int))


indx = Y_TRUES.argsort()
Y_TRUES = Y_TRUES[indx]
Y_PROBS = Y_PROBS[indx]
Y_PREDS = Y_PREDS[indx]


# Evaluation metrics
auc_roc = roc_auc_score(Y_TRUES, Y_PROBS)
precision, recall, _ = precision_recall_curve(Y_TRUES, Y_PROBS)
auc_pr = auc(recall, precision)
acc = accuracy_score(Y_TRUES, Y_PREDS)
pre = precision_score(Y_TRUES, Y_PREDS)
rec = recall_score(Y_TRUES, Y_PREDS)
f1 = f1_score(Y_TRUES, Y_PREDS)

# Confusion matrix: [ [TN, FP], [FN, TP] ]
tn, fp, fn, tp = confusion_matrix(Y_TRUES, Y_PREDS).ravel()

# Sensitivity = Recall = TP / (TP + FN)
sensitivity = tp / (tp + fn)

# Specificity = TN / (TN + FP)
specificity = tn / (tn + fp)

# Print results
print(f"AUC-ROC:    {auc_roc:.4f}")
print(f"AUC-PR:     {auc_pr:.4f}")
print(f"Accuracy:   {acc:.4f}")
print(f"Precision:  {pre:.4f}")
print(f"Recall:     {rec:.4f}")
print(f"F1 Score:   {f1:.4f}")
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity:          {specificity:.4f}")
print(confusion_matrix(Y_TRUES, Y_PREDS))


# plt.figure()
# plt.plot(Y_TRUES, 'o', color='blue', alpha=.25, markersize=8, label='Ground-truth')
# plt.plot(Y_PREDS, 'o', color='red', alpha=.25, markersize=8, label='Prediction')
# plt.show()

0 10779 11
1 10617 173
2 10488 302
3 10599 191
4 10604 186
5 10767 23
6 10437 353
7 10541 249
8 10500 290
9 10611 179
10 10771 19
11 10560 230
12 10532 258
13 10716 74
14 10524 266
15 10548 242
16 10665 125
17 10508 282
18 10668 122
19 10570 220
20 10588 202
21 10521 269
22 10627 163
23 10600 190
24 10771 19
25 10520 270
26 10495 295
27 10520 270
28 10605 185
29 10616 174
30 10576 214
31 10556 234
32 10687 103
33 10560 230
34 10625 165
35 10751 39
36 10525 265
37 10436 354
38 10621 169
39 10463 327
40 10524 266
41 10584 206
42 10632 158
43 10520 270
44 10728 62
45 10699 91
46 10763 27
47 10580 210
48 10672 118
49 10528 262
50 10627 163
51 10541 249
52 10502 288
53 10575 215
54 10532 258
55 10775 15
AUC-ROC:    0.9632
AUC-PR:     0.4177
Accuracy:   0.9869
Precision:  0.4786
Recall:     0.4963
F1 Score:   0.4873
Sensitivity (Recall): 0.4963
Specificity:          0.9931
[[10582    73]
 [   68    67]]


In [224]:
# Random Forest

Y_TRUES = np.empty([0])
Y_PROBS = np.empty([0])
Y_PREDS = np.empty([0])


cv = LeavePGroupsOut(n_groups=1)
for train_idx, test_idx in cv.split(x, y, groups=p):
    participant = np.unique(p[test_idx])[0]


# cv = KFold(n_splits=x.shape[0], shuffle=True, random_state=seed)
# for fold, (train_idx, test_idx) in enumerate(cv.split(x), start=1):
#     participant = np.unique(p[test_idx])[0]

# cv = KFold(n_splits=5, shuffle=True, random_state=seed)
# for fold, (train_idx, test_idx) in enumerate(cv.split(x), start=1):
#     participant = fold

    x_train, x_test = x[train_idx], x[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    print(participant, x_train.shape[0], x_test.shape[0])

    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)

    normalizer = MinMaxScaler()
    normalizer.fit(x_train)
    x_train = normalizer.transform(x_train)
    x_test = normalizer.transform(x_test)

    # Oversample class 1

    # ros = RandomOverSampler(sampling_strategy=.1, random_state=seed)
    # x_train, y_train = ros.fit_resample(x_train, y_train)

    # # 1. Check original class distribution
    # print("Original class distribution:", Counter(y_train))
    # # 2. Define the minority class (adjust if needed)
    # minority_class = 1  # change this if your minority class label is different
    # current_minority_count = sum(y_train == minority_class)
    # # 3. Define desired new total count for the minority class (10x)
    # target_minority_count = current_minority_count * 10
    # # 4. Setup SMOTE with custom sampling strategy
    # smote = SMOTE(sampling_strategy={minority_class: target_minority_count}, random_state=seed)
    # # 5. Fit and resample
    # x_train, y_train = smote.fit_resample(x_train, y_train)
    # # 6. Confirm new distribution
    # print("Resampled class distribution:", Counter(y_train))


    # ----- LightGBM
    # train_data = lgb.Dataset(x_train, label=y_train)
    # test_data = lgb.Dataset(x_test, label=y_test, reference=train_data)
    # params = {
    #     'verbose': -1,  # 👈 turn off training output
    #     'objective': 'binary',
    #     'metric': 'binary_logloss',  # or 'auc' if you prefer
    #     'num_leaves': 64,
    #     'learning_rate': 0.01,
    #     'n_estimators': 100,
    #     # 'is_unbalance': True  # Automatically balances positive and negative classes
    #     # 'scale_pos_weight': (y_train == 0).sum() / (y_train == 1).sum()
    # }
    # bst = lgb.train(params, train_data, valid_sets=[train_data, test_data])
    # y_probs = bst.predict(x_test, num_iteration=bst.best_iteration)


    # ----- Transformer
    # model = TabPFNClassifier(
    #     device='cuda',
    #     random_state=seed,
    #     n_estimators = 4,
    #     ignore_pretraining_limits=True)
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]


    # ----- Gradient Boosting Classifier
    # model = GradientBoostingClassifier(
    #     n_estimators=100,  # match LightGBM
    #     learning_rate=0.001,  # match LightGBM
    #     max_depth=4,  # similar to LightGBM default tree depth
    #     subsample=1.0,  # default
    #     random_state=seed
    # )
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]
    

    # ----- XGBoost
    # model = XGBClassifier()
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]


    # ----- Random Forest
    model = RandomForestClassifier(
        n_estimators=250,  # number of trees
        max_depth=None,  # let the trees grow fully
        random_state=seed
    )
    model.fit(x_train, y_train)
    y_probs = model.predict_proba(x_test)[:, 1]


    Y_TRUES = np.append(Y_TRUES, y_test)
    Y_PROBS = np.append(Y_PROBS, y_probs)
    Y_PREDS = np.append(Y_PREDS, (y_probs >= .5).astype(int))


indx = Y_TRUES.argsort()
Y_TRUES = Y_TRUES[indx]
Y_PROBS = Y_PROBS[indx]
Y_PREDS = Y_PREDS[indx]


# Evaluation metrics
auc_roc = roc_auc_score(Y_TRUES, Y_PROBS)
precision, recall, _ = precision_recall_curve(Y_TRUES, Y_PROBS)
auc_pr = auc(recall, precision)
acc = accuracy_score(Y_TRUES, Y_PREDS)
pre = precision_score(Y_TRUES, Y_PREDS)
rec = recall_score(Y_TRUES, Y_PREDS)
f1 = f1_score(Y_TRUES, Y_PREDS)

# Confusion matrix: [ [TN, FP], [FN, TP] ]
tn, fp, fn, tp = confusion_matrix(Y_TRUES, Y_PREDS).ravel()

# Sensitivity = Recall = TP / (TP + FN)
sensitivity = tp / (tp + fn)

# Specificity = TN / (TN + FP)
specificity = tn / (tn + fp)

# Print results
print(f"AUC-ROC:    {auc_roc:.4f}")
print(f"AUC-PR:     {auc_pr:.4f}")
print(f"Accuracy:   {acc:.4f}")
print(f"Precision:  {pre:.4f}")
print(f"Recall:     {rec:.4f}")
print(f"F1 Score:   {f1:.4f}")
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity:          {specificity:.4f}")
print(confusion_matrix(Y_TRUES, Y_PREDS))


# plt.figure()
# plt.plot(Y_TRUES, 'o', color='blue', alpha=.25, markersize=8, label='Ground-truth')
# plt.plot(Y_PREDS, 'o', color='red', alpha=.25, markersize=8, label='Prediction')
# plt.show()

0 10779 11
1 10617 173
2 10488 302
3 10599 191
4 10604 186
5 10767 23
6 10437 353
7 10541 249
8 10500 290
9 10611 179
10 10771 19
11 10560 230
12 10532 258
13 10716 74
14 10524 266
15 10548 242
16 10665 125
17 10508 282
18 10668 122
19 10570 220
20 10588 202
21 10521 269
22 10627 163
23 10600 190
24 10771 19
25 10520 270
26 10495 295
27 10520 270
28 10605 185
29 10616 174
30 10576 214
31 10556 234
32 10687 103
33 10560 230
34 10625 165
35 10751 39
36 10525 265
37 10436 354
38 10621 169
39 10463 327
40 10524 266
41 10584 206
42 10632 158
43 10520 270
44 10728 62
45 10699 91
46 10763 27
47 10580 210
48 10672 118
49 10528 262
50 10627 163
51 10541 249
52 10502 288
53 10575 215
54 10532 258
55 10775 15
AUC-ROC:    0.9270
AUC-PR:     0.3763
Accuracy:   0.9878
Precision:  0.5652
Recall:     0.0963
F1 Score:   0.1646
Sensitivity (Recall): 0.0963
Specificity:          0.9991
[[10645    10]
 [  122    13]]


In [225]:
# SVM

Y_TRUES = np.empty([0])
Y_PROBS = np.empty([0])
Y_PREDS = np.empty([0])


cv = LeavePGroupsOut(n_groups=1)
for train_idx, test_idx in cv.split(x, y, groups=p):
    participant = np.unique(p[test_idx])[0]


# cv = KFold(n_splits=x.shape[0], shuffle=True, random_state=seed)
# for fold, (train_idx, test_idx) in enumerate(cv.split(x), start=1):
#     participant = np.unique(p[test_idx])[0]

# cv = KFold(n_splits=5, shuffle=True, random_state=seed)
# for fold, (train_idx, test_idx) in enumerate(cv.split(x), start=1):
#     participant = fold

    x_train, x_test = x[train_idx], x[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    print(participant, x_train.shape[0], x_test.shape[0])

    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)

    normalizer = MinMaxScaler()
    normalizer.fit(x_train)
    x_train = normalizer.transform(x_train)
    x_test = normalizer.transform(x_test)

    # Oversample class 1

    # ros = RandomOverSampler(sampling_strategy=.1, random_state=seed)
    # x_train, y_train = ros.fit_resample(x_train, y_train)

    # # 1. Check original class distribution
    # print("Original class distribution:", Counter(y_train))
    # # 2. Define the minority class (adjust if needed)
    # minority_class = 1  # change this if your minority class label is different
    # current_minority_count = sum(y_train == minority_class)
    # # 3. Define desired new total count for the minority class (10x)
    # target_minority_count = current_minority_count * 10
    # # 4. Setup SMOTE with custom sampling strategy
    # smote = SMOTE(sampling_strategy={minority_class: target_minority_count}, random_state=seed)
    # # 5. Fit and resample
    # x_train, y_train = smote.fit_resample(x_train, y_train)
    # # 6. Confirm new distribution
    # print("Resampled class distribution:", Counter(y_train))


    # ----- LightGBM
    # train_data = lgb.Dataset(x_train, label=y_train)
    # test_data = lgb.Dataset(x_test, label=y_test, reference=train_data)
    # params = {
    #     'verbose': -1,  # 👈 turn off training output
    #     'objective': 'binary',
    #     'metric': 'binary_logloss',  # or 'auc' if you prefer
    #     'num_leaves': 64,
    #     'learning_rate': 0.01,
    #     'n_estimators': 100,
    #     # 'is_unbalance': True  # Automatically balances positive and negative classes
    #     # 'scale_pos_weight': (y_train == 0).sum() / (y_train == 1).sum()
    # }
    # bst = lgb.train(params, train_data, valid_sets=[train_data, test_data])
    # y_probs = bst.predict(x_test, num_iteration=bst.best_iteration)


    # ----- Transformer
    # model = TabPFNClassifier(
    #     device='cuda',
    #     random_state=seed,
    #     n_estimators = 4,
    #     ignore_pretraining_limits=True)
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]


    # ----- Gradient Boosting Classifier
    # model = GradientBoostingClassifier(
    #     n_estimators=100,  # match LightGBM
    #     learning_rate=0.001,  # match LightGBM
    #     max_depth=4,  # similar to LightGBM default tree depth
    #     subsample=1.0,  # default
    #     random_state=seed
    # )
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]
    

    # ----- XGBoost
    # model = XGBClassifier()
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]


    # ----- Random Forest
    # model = RandomForestClassifier(
    #     n_estimators=250,  # number of trees
    #     max_depth=None,  # let the trees grow fully
    #     random_state=seed
    # )
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]


    # ----- SVM
    model = SVC(kernel='rbf', probability=True, random_state=seed)
    model.fit(x_train, y_train)
    y_probs = model.predict_proba(x_test)[:, 1]




    Y_TRUES = np.append(Y_TRUES, y_test)
    Y_PROBS = np.append(Y_PROBS, y_probs)
    Y_PREDS = np.append(Y_PREDS, (y_probs >= .5).astype(int))


indx = Y_TRUES.argsort()
Y_TRUES = Y_TRUES[indx]
Y_PROBS = Y_PROBS[indx]
Y_PREDS = Y_PREDS[indx]


# Evaluation metrics
auc_roc = roc_auc_score(Y_TRUES, Y_PROBS)
precision, recall, _ = precision_recall_curve(Y_TRUES, Y_PROBS)
auc_pr = auc(recall, precision)
acc = accuracy_score(Y_TRUES, Y_PREDS)
pre = precision_score(Y_TRUES, Y_PREDS)
rec = recall_score(Y_TRUES, Y_PREDS)
f1 = f1_score(Y_TRUES, Y_PREDS)

# Confusion matrix: [ [TN, FP], [FN, TP] ]
tn, fp, fn, tp = confusion_matrix(Y_TRUES, Y_PREDS).ravel()

# Sensitivity = Recall = TP / (TP + FN)
sensitivity = tp / (tp + fn)

# Specificity = TN / (TN + FP)
specificity = tn / (tn + fp)

# Print results
print(f"AUC-ROC:    {auc_roc:.4f}")
print(f"AUC-PR:     {auc_pr:.4f}")
print(f"Accuracy:   {acc:.4f}")
print(f"Precision:  {pre:.4f}")
print(f"Recall:     {rec:.4f}")
print(f"F1 Score:   {f1:.4f}")
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity:          {specificity:.4f}")
print(confusion_matrix(Y_TRUES, Y_PREDS))


# plt.figure()
# plt.plot(Y_TRUES, 'o', color='blue', alpha=.25, markersize=8, label='Ground-truth')
# plt.plot(Y_PREDS, 'o', color='red', alpha=.25, markersize=8, label='Prediction')
# plt.show()

0 10779 11
1 10617 173
2 10488 302
3 10599 191
4 10604 186
5 10767 23
6 10437 353
7 10541 249
8 10500 290
9 10611 179
10 10771 19
11 10560 230
12 10532 258
13 10716 74
14 10524 266
15 10548 242
16 10665 125
17 10508 282
18 10668 122
19 10570 220
20 10588 202
21 10521 269
22 10627 163
23 10600 190
24 10771 19
25 10520 270
26 10495 295
27 10520 270
28 10605 185
29 10616 174
30 10576 214
31 10556 234
32 10687 103
33 10560 230
34 10625 165
35 10751 39
36 10525 265
37 10436 354
38 10621 169
39 10463 327
40 10524 266
41 10584 206
42 10632 158
43 10520 270
44 10728 62
45 10699 91
46 10763 27
47 10580 210
48 10672 118
49 10528 262
50 10627 163
51 10541 249
52 10502 288
53 10575 215
54 10532 258
55 10775 15
AUC-ROC:    0.8021
AUC-PR:     0.1697
Accuracy:   0.9860
Precision:  0.2143
Recall:     0.0444
F1 Score:   0.0736
Sensitivity (Recall): 0.0444
Specificity:          0.9979
[[10633    22]
 [  129     6]]


In [226]:
# DT

Y_TRUES = np.empty([0])
Y_PROBS = np.empty([0])
Y_PREDS = np.empty([0])


cv = LeavePGroupsOut(n_groups=1)
for train_idx, test_idx in cv.split(x, y, groups=p):
    participant = np.unique(p[test_idx])[0]


# cv = KFold(n_splits=x.shape[0], shuffle=True, random_state=seed)
# for fold, (train_idx, test_idx) in enumerate(cv.split(x), start=1):
#     participant = np.unique(p[test_idx])[0]

# cv = KFold(n_splits=5, shuffle=True, random_state=seed)
# for fold, (train_idx, test_idx) in enumerate(cv.split(x), start=1):
#     participant = fold

    x_train, x_test = x[train_idx], x[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    print(participant, x_train.shape[0], x_test.shape[0])

    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)

    normalizer = MinMaxScaler()
    normalizer.fit(x_train)
    x_train = normalizer.transform(x_train)
    x_test = normalizer.transform(x_test)

    # Oversample class 1

    # ros = RandomOverSampler(sampling_strategy=.1, random_state=seed)
    # x_train, y_train = ros.fit_resample(x_train, y_train)

    # # 1. Check original class distribution
    # print("Original class distribution:", Counter(y_train))
    # # 2. Define the minority class (adjust if needed)
    # minority_class = 1  # change this if your minority class label is different
    # current_minority_count = sum(y_train == minority_class)
    # # 3. Define desired new total count for the minority class (10x)
    # target_minority_count = current_minority_count * 10
    # # 4. Setup SMOTE with custom sampling strategy
    # smote = SMOTE(sampling_strategy={minority_class: target_minority_count}, random_state=seed)
    # # 5. Fit and resample
    # x_train, y_train = smote.fit_resample(x_train, y_train)
    # # 6. Confirm new distribution
    # print("Resampled class distribution:", Counter(y_train))


    # ----- LightGBM
    # train_data = lgb.Dataset(x_train, label=y_train)
    # test_data = lgb.Dataset(x_test, label=y_test, reference=train_data)
    # params = {
    #     'verbose': -1,  # 👈 turn off training output
    #     'objective': 'binary',
    #     'metric': 'binary_logloss',  # or 'auc' if you prefer
    #     'num_leaves': 64,
    #     'learning_rate': 0.01,
    #     'n_estimators': 100,
    #     # 'is_unbalance': True  # Automatically balances positive and negative classes
    #     # 'scale_pos_weight': (y_train == 0).sum() / (y_train == 1).sum()
    # }
    # bst = lgb.train(params, train_data, valid_sets=[train_data, test_data])
    # y_probs = bst.predict(x_test, num_iteration=bst.best_iteration)


    # ----- Transformer
    # model = TabPFNClassifier(
    #     device='cuda',
    #     random_state=seed,
    #     n_estimators = 4,
    #     ignore_pretraining_limits=True)
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]


    # ----- Gradient Boosting Classifier
    # model = GradientBoostingClassifier(
    #     n_estimators=100,  # match LightGBM
    #     learning_rate=0.001,  # match LightGBM
    #     max_depth=4,  # similar to LightGBM default tree depth
    #     subsample=1.0,  # default
    #     random_state=seed
    # )
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]
    

    # ----- XGBoost
    # model = XGBClassifier()
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]


    # ----- Random Forest
    # model = RandomForestClassifier(
    #     n_estimators=250,  # number of trees
    #     max_depth=None,  # let the trees grow fully
    #     random_state=seed
    # )
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]


    # ----- SVM
    # model = SVC(kernel='rbf', probability=True, random_state=seed)
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]


    # ----- DT
    model = DecisionTreeClassifier(criterion="log_loss", max_depth=512, random_state=seed)
    model.fit(x_train, y_train)
    y_probs = model.predict(x_test)



    Y_TRUES = np.append(Y_TRUES, y_test)
    Y_PROBS = np.append(Y_PROBS, y_probs)
    Y_PREDS = np.append(Y_PREDS, (y_probs >= .5).astype(int))


indx = Y_TRUES.argsort()
Y_TRUES = Y_TRUES[indx]
Y_PROBS = Y_PROBS[indx]
Y_PREDS = Y_PREDS[indx]


# Evaluation metrics
auc_roc = roc_auc_score(Y_TRUES, Y_PROBS)
precision, recall, _ = precision_recall_curve(Y_TRUES, Y_PROBS)
auc_pr = auc(recall, precision)
acc = accuracy_score(Y_TRUES, Y_PREDS)
pre = precision_score(Y_TRUES, Y_PREDS)
rec = recall_score(Y_TRUES, Y_PREDS)
f1 = f1_score(Y_TRUES, Y_PREDS)

# Confusion matrix: [ [TN, FP], [FN, TP] ]
tn, fp, fn, tp = confusion_matrix(Y_TRUES, Y_PREDS).ravel()

# Sensitivity = Recall = TP / (TP + FN)
sensitivity = tp / (tp + fn)

# Specificity = TN / (TN + FP)
specificity = tn / (tn + fp)

# Print results
print(f"AUC-ROC:    {auc_roc:.4f}")
print(f"AUC-PR:     {auc_pr:.4f}")
print(f"Accuracy:   {acc:.4f}")
print(f"Precision:  {pre:.4f}")
print(f"Recall:     {rec:.4f}")
print(f"F1 Score:   {f1:.4f}")
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity:          {specificity:.4f}")
print(confusion_matrix(Y_TRUES, Y_PREDS))


# plt.figure()
# plt.plot(Y_TRUES, 'o', color='blue', alpha=.25, markersize=8, label='Ground-truth')
# plt.plot(Y_PREDS, 'o', color='red', alpha=.25, markersize=8, label='Prediction')
# plt.show()

0 10779 11
1 10617 173
2 10488 302
3 10599 191
4 10604 186
5 10767 23
6 10437 353
7 10541 249
8 10500 290
9 10611 179
10 10771 19
11 10560 230
12 10532 258
13 10716 74
14 10524 266
15 10548 242
16 10665 125
17 10508 282
18 10668 122
19 10570 220
20 10588 202
21 10521 269
22 10627 163
23 10600 190
24 10771 19
25 10520 270
26 10495 295
27 10520 270
28 10605 185
29 10616 174
30 10576 214
31 10556 234
32 10687 103
33 10560 230
34 10625 165
35 10751 39
36 10525 265
37 10436 354
38 10621 169
39 10463 327
40 10524 266
41 10584 206
42 10632 158
43 10520 270
44 10728 62
45 10699 91
46 10763 27
47 10580 210
48 10672 118
49 10528 262
50 10627 163
51 10541 249
52 10502 288
53 10575 215
54 10532 258
55 10775 15
AUC-ROC:    0.6371
AUC-PR:     0.3090
Accuracy:   0.9838
Precision:  0.3276
Recall:     0.2815
F1 Score:   0.3028
Sensitivity (Recall): 0.2815
Specificity:          0.9927
[[10577    78]
 [   97    38]]


In [227]:
# MLP

Y_TRUES = np.empty([0])
Y_PROBS = np.empty([0])
Y_PREDS = np.empty([0])


cv = LeavePGroupsOut(n_groups=1)
for train_idx, test_idx in cv.split(x, y, groups=p):
    participant = np.unique(p[test_idx])[0]


# cv = KFold(n_splits=x.shape[0], shuffle=True, random_state=seed)
# for fold, (train_idx, test_idx) in enumerate(cv.split(x), start=1):
#     participant = np.unique(p[test_idx])[0]

# cv = KFold(n_splits=5, shuffle=True, random_state=seed)
# for fold, (train_idx, test_idx) in enumerate(cv.split(x), start=1):
#     participant = fold

    x_train, x_test = x[train_idx], x[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    print(participant, x_train.shape[0], x_test.shape[0])

    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)

    normalizer = MinMaxScaler()
    normalizer.fit(x_train)
    x_train = normalizer.transform(x_train)
    x_test = normalizer.transform(x_test)

    # Oversample class 1

    # ros = RandomOverSampler(sampling_strategy=.1, random_state=seed)
    # x_train, y_train = ros.fit_resample(x_train, y_train)

    # # 1. Check original class distribution
    # print("Original class distribution:", Counter(y_train))
    # # 2. Define the minority class (adjust if needed)
    # minority_class = 1  # change this if your minority class label is different
    # current_minority_count = sum(y_train == minority_class)
    # # 3. Define desired new total count for the minority class (10x)
    # target_minority_count = current_minority_count * 10
    # # 4. Setup SMOTE with custom sampling strategy
    # smote = SMOTE(sampling_strategy={minority_class: target_minority_count}, random_state=seed)
    # # 5. Fit and resample
    # x_train, y_train = smote.fit_resample(x_train, y_train)
    # # 6. Confirm new distribution
    # print("Resampled class distribution:", Counter(y_train))


    # ----- LightGBM
    # train_data = lgb.Dataset(x_train, label=y_train)
    # test_data = lgb.Dataset(x_test, label=y_test, reference=train_data)
    # params = {
    #     'verbose': -1,  # 👈 turn off training output
    #     'objective': 'binary',
    #     'metric': 'binary_logloss',  # or 'auc' if you prefer
    #     'num_leaves': 64,
    #     'learning_rate': 0.01,
    #     'n_estimators': 100,
    #     # 'is_unbalance': True  # Automatically balances positive and negative classes
    #     # 'scale_pos_weight': (y_train == 0).sum() / (y_train == 1).sum()
    # }
    # bst = lgb.train(params, train_data, valid_sets=[train_data, test_data])
    # y_probs = bst.predict(x_test, num_iteration=bst.best_iteration)


    # ----- Transformer
    # model = TabPFNClassifier(
    #     device='cuda',
    #     random_state=seed,
    #     n_estimators = 4,
    #     ignore_pretraining_limits=True)
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]


    # ----- Gradient Boosting Classifier
    # model = GradientBoostingClassifier(
    #     n_estimators=100,  # match LightGBM
    #     learning_rate=0.001,  # match LightGBM
    #     max_depth=4,  # similar to LightGBM default tree depth
    #     subsample=1.0,  # default
    #     random_state=seed
    # )
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]
    

    # ----- XGBoost
    # model = XGBClassifier()
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]


    # ----- Random Forest
    # model = RandomForestClassifier(
    #     n_estimators=250,  # number of trees
    #     max_depth=None,  # let the trees grow fully
    #     random_state=seed
    # )
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]


    # ----- SVM
    # model = SVC(kernel='rbf', probability=True, random_state=seed)
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]


    # ----- DT
    # model = DecisionTreeClassifier(criterion="log_loss", max_depth=512, random_state=seed)
    # model.fit(x_train, y_train)
    # y_probs = model.predict(x_test)


    # MLP
    model = MLPClassifier(
        hidden_layer_sizes=(128, 64),    # 2 hidden layers: 64 and 32 neurons
        activation='relu',              # good default: 'relu'
        solver='adam',                  # optimizer
        alpha=0.0001,                   # L2 regularization
        learning_rate_init=0.001,
        max_iter=500,
        early_stopping=True,
        random_state=seed,
        verbose=True,
    )
    model.fit(x_train, y_train)    
    y_probs = model.predict_proba(x_test)[:, 1]
    y_preds = model.predict(x_test)




    Y_TRUES = np.append(Y_TRUES, y_test)
    Y_PROBS = np.append(Y_PROBS, y_probs)
    Y_PREDS = np.append(Y_PREDS, y_preds)


indx = Y_TRUES.argsort()
Y_TRUES = Y_TRUES[indx]
Y_PROBS = Y_PROBS[indx]
Y_PREDS = Y_PREDS[indx]


# Evaluation metrics
auc_roc = roc_auc_score(Y_TRUES, Y_PROBS)
precision, recall, _ = precision_recall_curve(Y_TRUES, Y_PROBS)
auc_pr = auc(recall, precision)
acc = accuracy_score(Y_TRUES, Y_PREDS)
pre = precision_score(Y_TRUES, Y_PREDS)
rec = recall_score(Y_TRUES, Y_PREDS)
f1 = f1_score(Y_TRUES, Y_PREDS)

# Confusion matrix: [ [TN, FP], [FN, TP] ]
tn, fp, fn, tp = confusion_matrix(Y_TRUES, Y_PREDS).ravel()

# Sensitivity = Recall = TP / (TP + FN)
sensitivity = tp / (tp + fn)

# Specificity = TN / (TN + FP)
specificity = tn / (tn + fp)

# Print results
print(f"AUC-ROC:    {auc_roc:.4f}")
print(f"AUC-PR:     {auc_pr:.4f}")
print(f"Accuracy:   {acc:.4f}")
print(f"Precision:  {pre:.4f}")
print(f"Recall:     {rec:.4f}")
print(f"F1 Score:   {f1:.4f}")
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity:          {specificity:.4f}")
print(confusion_matrix(Y_TRUES, Y_PREDS))


# plt.figure()
# plt.plot(Y_TRUES, 'o', color='blue', alpha=.25, markersize=8, label='Ground-truth')
# plt.plot(Y_PREDS, 'o', color='red', alpha=.25, markersize=8, label='Prediction')
# plt.show()

0 10779 11
Iteration 1, loss = 0.22359075
Validation score: 0.987013
Iteration 2, loss = 0.07409667
Validation score: 0.987013
Iteration 3, loss = 0.05419803
Validation score: 0.987013
Iteration 4, loss = 0.04736082
Validation score: 0.987013
Iteration 5, loss = 0.04526200
Validation score: 0.987013
Iteration 6, loss = 0.04457983
Validation score: 0.987013
Iteration 7, loss = 0.04392281
Validation score: 0.987013
Iteration 8, loss = 0.04302360
Validation score: 0.987013
Iteration 9, loss = 0.04293123
Validation score: 0.987013
Iteration 10, loss = 0.04338872
Validation score: 0.987013
Iteration 11, loss = 0.04240246
Validation score: 0.987013
Iteration 12, loss = 0.04298408
Validation score: 0.987013
Validation score did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
1 10617 173
Iteration 1, loss = 0.22521033
Validation score: 0.987759
Iteration 2, loss = 0.07557132
Validation score: 0.987759
Iteration 3, loss = 0.05631219
Validation score: 0.987759
Iteration 4

In [56]:
# Logistic Regression

Y_TRUES = np.empty([0])
Y_PROBS = np.empty([0])
Y_PREDS = np.empty([0])


# cv = LeavePGroupsOut(n_groups=1)
# for train_idx, test_idx in cv.split(x, y, groups=p):
#     participant = np.unique(p[test_idx])[0]


# cv = KFold(n_splits=x.shape[0], shuffle=True, random_state=seed)
# for fold, (train_idx, test_idx) in enumerate(cv.split(x), start=1):
#     participant = np.unique(p[test_idx])[0]

cv = KFold(n_splits=5, shuffle=True, random_state=seed)
for fold, (train_idx, test_idx) in enumerate(cv.split(x), start=1):
    participant = fold

    x_train, x_test = x[train_idx], x[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    print(participant, x_train.shape[0], x_test.shape[0])

    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)

    normalizer = MinMaxScaler()
    normalizer.fit(x_train)
    x_train = normalizer.transform(x_train)
    x_test = normalizer.transform(x_test)

    # Oversample class 1

    # ros = RandomOverSampler(sampling_strategy=.1, random_state=seed)
    # x_train, y_train = ros.fit_resample(x_train, y_train)

    # 1. Check original class distribution
    # print("Original class distribution:", Counter(y_train))
    # # 2. Define the minority class (adjust if needed)
    # minority_class = 1  # change this if your minority class label is different
    # current_minority_count = sum(y_train == minority_class)
    # # 3. Define desired new total count for the minority class (10x)
    # target_minority_count = current_minority_count * 10
    # # 4. Setup SMOTE with custom sampling strategy
    # smote = SMOTE(sampling_strategy={minority_class: target_minority_count}, random_state=seed)
    # # 5. Fit and resample
    # x_train, y_train = smote.fit_resample(x_train, y_train)
    # # 6. Confirm new distribution
    # print("Resampled class distribution:", Counter(y_train))


    # Undersample class 0
    # reduction_percent = 99
    # print("Original class distribution:", Counter(y_train))    
    # class_0_idx = np.where(y_train == 0)[0]
    # class_1_idx = np.where(y_train == 1)[0]
    # # How many class 0 samples to keep
    # n_keep = int(len(class_0_idx) * (1 - reduction_percent / 100.0))
    # sampled_0_idx = np.random.choice(class_0_idx, size=n_keep, replace=False)
    # # Combine sampled class 0 with all class 1
    # final_idx = np.concatenate([sampled_0_idx, class_1_idx])
    # x_train = x_train[final_idx]
    # y_train =  y_train[final_idx]
    # print("Resampled class distribution:", Counter(y_train))


    # ----- LightGBM
    # train_data = lgb.Dataset(x_train, label=y_train)
    # test_data = lgb.Dataset(x_test, label=y_test, reference=train_data)
    # params = {
    #     'verbose': -1,  # 👈 turn off training output
    #     'objective': 'binary',
    #     'metric': 'binary_logloss',  # or 'auc' if you prefer
    #     'num_leaves': 64,
    #     'learning_rate': 0.01,
    #     'n_estimators': 100,
    #     # 'is_unbalance': True  # Automatically balances positive and negative classes
    #     # 'scale_pos_weight': (y_train == 0).sum() / (y_train == 1).sum()
    # }
    # bst = lgb.train(params, train_data, valid_sets=[train_data, test_data])
    # y_probs = bst.predict(x_test, num_iteration=bst.best_iteration)


    # ----- Transformer
    # model = TabPFNClassifier(
    #     device='cuda',
    #     random_state=seed,
    #     n_estimators = 4,
    #     ignore_pretraining_limits=True)
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]


    # ----- Gradient Boosting Classifier
    # model = GradientBoostingClassifier(
    #     n_estimators=100,  # match LightGBM
    #     learning_rate=0.001,  # match LightGBM
    #     max_depth=4,  # similar to LightGBM default tree depth
    #     subsample=1.0,  # default
    #     random_state=seed
    # )
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]
    

    # ----- XGBoost
    # model = XGBClassifier()
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]


    # ----- Random Forest
    # model = RandomForestClassifier(
    #     n_estimators=250,  # number of trees
    #     max_depth=None,  # let the trees grow fully
    #     random_state=seed
    # )
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]


    # ----- SVM
    # model = SVC(kernel='rbf', probability=True, random_state=seed)
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]


    # ----- DT
    # model = DecisionTreeClassifier(criterion="log_loss", max_depth=512, random_state=seed)
    # model.fit(x_train, y_train)
    # y_probs = model.predict(x_test)


    # MLP
    # model = MLPClassifier(
    #     hidden_layer_sizes=(128, 64),    # 2 hidden layers: 64 and 32 neurons
    #     activation='relu',              # good default: 'relu'
    #     solver='adam',                  # optimizer
    #     alpha=0.0001,                   # L2 regularization
    #     learning_rate_init=0.001,
    #     max_iter=500,
    #     early_stopping=True,
    #     random_state=seed,
    #     verbose=True,
    # )
    # model.fit(x_train, y_train)    
    # y_probs = model.predict_proba(x_test)[:, 1]
    # y_preds = model.predict(x_test)


    # LR
    model = LogisticRegression(
    penalty='l2',               # regularization (default)
    C=1.0,                      # inverse of regularization strength
    solver='lbfgs',             # optimizer (good default for small/medium data)
    max_iter=1000,
    class_weight='balanced',
    random_state=seed)
    model.fit(x_train, y_train)
    y_probs = model.predict_proba(x_test)[:, 1]
    y_preds = model.predict(x_test)



    Y_TRUES = np.append(Y_TRUES, y_test)
    Y_PROBS = np.append(Y_PROBS, y_probs)
    Y_PREDS = np.append(Y_PREDS, y_preds)


indx = Y_TRUES.argsort()
Y_TRUES = Y_TRUES[indx]
Y_PROBS = Y_PROBS[indx]
Y_PREDS = Y_PREDS[indx]


# Evaluation metrics
auc_roc = roc_auc_score(Y_TRUES, Y_PROBS)
precision, recall, _ = precision_recall_curve(Y_TRUES, Y_PROBS)
auc_pr = auc(recall, precision)
acc = accuracy_score(Y_TRUES, Y_PREDS)
pre = precision_score(Y_TRUES, Y_PREDS)
rec = recall_score(Y_TRUES, Y_PREDS)
f1 = f1_score(Y_TRUES, Y_PREDS)

# Confusion matrix: [ [TN, FP], [FN, TP] ]
tn, fp, fn, tp = confusion_matrix(Y_TRUES, Y_PREDS).ravel()

# Sensitivity = Recall = TP / (TP + FN)
sensitivity = tp / (tp + fn)

# Specificity = TN / (TN + FP)
specificity = tn / (tn + fp)

# Print results
print(f"AUC-ROC:    {auc_roc:.4f}")
print(f"AUC-PR:     {auc_pr:.4f}")
print(f"Accuracy:   {acc:.4f}")
print(f"Precision:  {pre:.4f}")
print(f"Recall:     {rec:.4f}")
print(f"F1 Score:   {f1:.4f}")
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity:          {specificity:.4f}")
print(confusion_matrix(Y_TRUES, Y_PREDS))


# plt.figure()
# plt.plot(Y_TRUES, 'o', color='blue', alpha=.25, markersize=8, label='Ground-truth')
# plt.plot(Y_PREDS, 'o', color='red', alpha=.25, markersize=8, label='Prediction')
# plt.show()

1 8632 2158
2 8632 2158
3 8632 2158
4 8632 2158
5 8632 2158
AUC-ROC:    0.9154
AUC-PR:     0.2614
Accuracy:   0.8911
Precision:  0.0860
Recall:     0.8000
F1 Score:   0.1553
Sensitivity (Recall): 0.8000
Specificity:          0.8923
[[9507 1148]
 [  27  108]]


In [28]:
# Naive Bayes

Y_TRUES = np.empty([0])
Y_PROBS = np.empty([0])
Y_PREDS = np.empty([0])


# cv = LeavePGroupsOut(n_groups=1)
# for train_idx, test_idx in cv.split(x, y, groups=p):
#     participant = np.unique(p[test_idx])[0]


# cv = KFold(n_splits=x.shape[0], shuffle=True, random_state=seed)
# for fold, (train_idx, test_idx) in enumerate(cv.split(x), start=1):
#     participant = np.unique(p[test_idx])[0]

cv = KFold(n_splits=5, shuffle=True, random_state=seed)
for fold, (train_idx, test_idx) in enumerate(cv.split(x), start=1):
    participant = fold

    x_train, x_test = x[train_idx], x[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    print(fold, participant, x_train.shape[0], x_test.shape[0])

    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)

    normalizer = MinMaxScaler()
    normalizer.fit(x_train)
    x_train = normalizer.transform(x_train)
    x_test = normalizer.transform(x_test)

    # Oversample class 1

    # ros = RandomOverSampler(sampling_strategy=.1, random_state=seed)
    # x_train, y_train = ros.fit_resample(x_train, y_train)

    # # 1. Check original class distribution
    # print("Original class distribution:", Counter(y_train))
    # # 2. Define the minority class (adjust if needed)
    # minority_class = 1  # change this if your minority class label is different
    # current_minority_count = sum(y_train == minority_class)
    # # 3. Define desired new total count for the minority class (10x)
    # target_minority_count = current_minority_count * 10
    # # 4. Setup SMOTE with custom sampling strategy
    # smote = SMOTE(sampling_strategy={minority_class: target_minority_count}, random_state=seed)
    # # 5. Fit and resample
    # x_train, y_train = smote.fit_resample(x_train, y_train)
    # # 6. Confirm new distribution
    # print("Resampled class distribution:", Counter(y_train))


    # Undersample class 0
    # reduction_percent = 99
    # print("Original class distribution:", Counter(y_train))    
    # class_0_idx = np.where(y_train == 0)[0]
    # class_1_idx = np.where(y_train == 1)[0]
    # # How many class 0 samples to keep
    # n_keep = int(len(class_0_idx) * (1 - reduction_percent / 100.0))
    # sampled_0_idx = np.random.choice(class_0_idx, size=n_keep, replace=False)
    # # Combine sampled class 0 with all class 1
    # final_idx = np.concatenate([sampled_0_idx, class_1_idx])
    # x_train = x_train[final_idx]
    # y_train =  y_train[final_idx]
    # print("Resampled class distribution:", Counter(y_train))


    # ----- LightGBM
    # train_data = lgb.Dataset(x_train, label=y_train)
    # test_data = lgb.Dataset(x_test, label=y_test, reference=train_data)
    # params = {
    #     'verbose': -1,  # 👈 turn off training output
    #     'objective': 'binary',
    #     'metric': 'binary_logloss',  # or 'auc' if you prefer
    #     'num_leaves': 64,
    #     'learning_rate': 0.01,
    #     'n_estimators': 100,
    #     # 'is_unbalance': True  # Automatically balances positive and negative classes
    #     # 'scale_pos_weight': (y_train == 0).sum() / (y_train == 1).sum()
    # }
    # bst = lgb.train(params, train_data, valid_sets=[train_data, test_data])
    # y_probs = bst.predict(x_test, num_iteration=bst.best_iteration)


    # ----- Transformer
    # model = TabPFNClassifier(
    #     device='cuda',
    #     random_state=seed,
    #     n_estimators = 4,
    #     ignore_pretraining_limits=True)
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]


    # ----- Gradient Boosting Classifier
    # model = GradientBoostingClassifier(
    #     n_estimators=100,  # match LightGBM
    #     learning_rate=0.001,  # match LightGBM
    #     max_depth=4,  # similar to LightGBM default tree depth
    #     subsample=1.0,  # default
    #     random_state=seed
    # )
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]
    

    # ----- XGBoost
    # model = XGBClassifier()
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]


    # ----- Random Forest
    # model = RandomForestClassifier(
    #     n_estimators=250,  # number of trees
    #     max_depth=None,  # let the trees grow fully
    #     random_state=seed
    # )
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]


    # ----- SVM
    # model = SVC(kernel='rbf', probability=True, random_state=seed)
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]


    # ----- DT
    # model = DecisionTreeClassifier(criterion="log_loss", max_depth=512, random_state=seed)
    # model.fit(x_train, y_train)
    # y_probs = model.predict(x_test)


    # MLP
    # model = MLPClassifier(
    #     hidden_layer_sizes=(128, 64),    # 2 hidden layers: 64 and 32 neurons
    #     activation='relu',              # good default: 'relu'
    #     solver='adam',                  # optimizer
    #     alpha=0.0001,                   # L2 regularization
    #     learning_rate_init=0.001,
    #     max_iter=500,
    #     early_stopping=True,
    #     random_state=seed,
    #     verbose=True,
    # )
    # model.fit(x_train, y_train)    
    # y_probs = model.predict_proba(x_test)[:, 1]
    # y_preds = model.predict(x_test)


    # LR
    # model = LogisticRegression(
    # penalty='l2',               # regularization (default)
    # C=1.0,                      # inverse of regularization strength
    # solver='lbfgs',             # optimizer (good default for small/medium data)
    # max_iter=1000,
    # class_weight='balanced',
    # random_state=seed)
    # model.fit(x_train, y_train)    
    # y_probs = model.predict_proba(x_test)[:, 1]
    # y_preds = model.predict(x_test)


    # NB
    model = GaussianNB()
    model.fit(x_train,
              y_train,
              sample_weight=compute_sample_weight(class_weight='balanced', y=y_train),
              )
    y_probs = model.predict_proba(x_test)[:, 1]
    y_preds = model.predict(x_test)



    Y_TRUES = np.append(Y_TRUES, y_test)
    Y_PROBS = np.append(Y_PROBS, y_probs)
    Y_PREDS = np.append(Y_PREDS, y_preds)


indx = Y_TRUES.argsort()
Y_TRUES = Y_TRUES[indx]
Y_PROBS = Y_PROBS[indx]
Y_PREDS = Y_PREDS[indx]


# Evaluation metrics
auc_roc = roc_auc_score(Y_TRUES, Y_PROBS)
precision, recall, _ = precision_recall_curve(Y_TRUES, Y_PROBS)
auc_pr = auc(recall, precision)
acc = accuracy_score(Y_TRUES, Y_PREDS)
pre = precision_score(Y_TRUES, Y_PREDS)
rec = recall_score(Y_TRUES, Y_PREDS)
f1 = f1_score(Y_TRUES, Y_PREDS)

# Confusion matrix: [ [TN, FP], [FN, TP] ]
tn, fp, fn, tp = confusion_matrix(Y_TRUES, Y_PREDS).ravel()

# Sensitivity = Recall = TP / (TP + FN)
sensitivity = tp / (tp + fn)

# Specificity = TN / (TN + FP)
specificity = tn / (tn + fp)

# Print results
print(f"AUC-ROC:    {auc_roc:.4f}")
print(f"AUC-PR:     {auc_pr:.4f}")
print(f"Accuracy:   {acc:.4f}")
print(f"Precision:  {pre:.4f}")
print(f"Recall:     {rec:.4f}")
print(f"F1 Score:   {f1:.4f}")
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity:          {specificity:.4f}")
print(confusion_matrix(Y_TRUES, Y_PREDS))


# plt.figure()
# plt.plot(Y_TRUES, 'o', color='blue', alpha=.25, markersize=8, label='Ground-truth')
# plt.plot(Y_PREDS, 'o', color='red', alpha=.25, markersize=8, label='Prediction')
# plt.show()

1 1 8632 2158
2 2 8632 2158
3 3 8632 2158
4 4 8632 2158
5 5 8632 2158
AUC-ROC:    0.8988
AUC-PR:     0.4455
Accuracy:   0.7080
Precision:  0.0374
Recall:     0.9037
F1 Score:   0.0719
Sensitivity (Recall): 0.9037
Specificity:          0.7055
[[7517 3138]
 [  13  122]]
